In [50]:
!pip install opencv-python
import numpy as np
import cv2
import time 
!pip install -U autopy
import autopy

In [51]:
#moving mouse coordinates
def move_mouse_by_defference(x_new,y_new):
    x0,y0=autopy.mouse.location()
    autopy.mouse.move(x0+x_new,y0+y_new)
move_mouse_by_defference(-30,150)

In [53]:
#Holds down or releases the given mouse button in the current position.
#Button can be LEFT, RIGHT, MIDDLE, or None to default to the left button.
def click_mouse_left():
    autopy.mouse.click(autopy.mouse.Button.LEFT)
click_mouse_left()

In [58]:
def click_mouse_right():
    autopy.mouse.click(button=autopy.mouse.Button.RIGHT)
click_mouse_right()